### Feature Engineering

In [65]:
import os, io, json, warnings
from datetime import datetime
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from dotenv import load_dotenv

import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
import joblib
from pathlib import Path

In [66]:
path = Path.cwd().parent / "dataset" /"processed" /"preprocessed_data.csv"
print(path)

/Users/haseebsagheer/Documents/Python Learning/Cloud-Retail-Insights/dataset/processed/preprocessed_data.csv


In [67]:
df_base = pd.read_csv(path)
df = df_base
pd.set_option("display.max_column",None)

df.head()


,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,DaysToShip,OrderYear,OrderMonth,OrderQuarter,OrderYearMonth,OrderWeekOfYear,OrderMonthName,OrderIsWeekendOrder,ShipYear,ShipMonth,ShipQuarter,ShipYearMonth,ShipWeekOfYear,ShipMonthName,ShipIsWeekendShip
0,ca-2017-152156,2017-11-08,2017-11-11,second class,cg-12520,claire gute,consumer,united states,henderson,kentucky,42420,south,FUR-BO-10001798,furniture,bookcases,bush somerset collection bookcase,261.9600,3,2017,11,4,2017-11,45,November,False,2017,11,4,2017-11,45,November,True
1,ca-2017-152156,2017-11-08,2017-11-11,second class,cg-12520,claire gute,consumer,united states,henderson,kentucky,42420,south,FUR-CH-10000454,furniture,chairs,"hon deluxe fabric upholstered stacking chairs,...",731.9400,3,2017,11,4,2017-11,45,November,False,2017,11,4,2017-11,45,November,True
2,ca-2017-138688,2017-06-12,2017-06-16,second class,dv-13045,darrin van huff,corporate,united states,los angeles,california,90036,west,OFF-LA-10000240,office supplies,labels,self-adhesive address labels for typewriters b...,14.6200,4,2017,6,2,2017-06,24,June,False,2017,6,2,2017-06,24,June,False
3,us-2016-108966,2016-10-11,2016-10-18,standard class,so-20335,sean o'donnell,consumer,united states,fort lauderdale,florida,33311,south,FUR-TA-10000577,furniture,tables,bretford cr4500 series slim rectangular table,957.5775,7,2016,10,4,2016-10,41,October,False,2016,10,4,2016-10,42,October,False
4,us-2016-108966,2016-10-11,2016-10-18,standard class,so-20335,sean o'donnell,consumer,united states,fort lauderdale,florida,33311,south,OFF-ST-10000760,office supplies,storage,eldon fold 'n roll cart system,22.3680,7,2016,10,4,2016-10,41,October,False,2016,10,4,2016-10,42,October,False


In [68]:

pd.set_option("display.max_row",None)
print(df.columns)


Index(['OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'Sales', 'DaysToShip', 'OrderYear', 'OrderMonth', 'OrderQuarter',
       'OrderYearMonth', 'OrderWeekOfYear', 'OrderMonthName',
       'OrderIsWeekendOrder', 'ShipYear', 'ShipMonth', 'ShipQuarter',
       'ShipYearMonth', 'ShipWeekOfYear', 'ShipMonthName',
       'ShipIsWeekendShip'],
      dtype='object')


In [ ]:
conn_url ="mssql+pyodbc:///?odbc_connect=DRIVER%3D%2Fopt%2Fhomebrew%2Flib%2Flibmsodbcsql.18.dylib%3BSERVER%3Dsqlsrv-retail-dev.database.windows.net%3BDATABASE%3Dsqldb-dretail-dev%3BUID%3Dsqladmin%3BPWD%3DHASEEBSAGHEER12%21%40%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dno%3BConnection+Timeout%3D30%3B"
